In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb

To make things faster, we're gonna use this notebook to train our xgboost models.

In [2]:
train = pd.read_csv("Darrel_Dataset/train_agg_fe_domain.csv")
test = pd.read_csv("Darrel_Dataset/test_agg_fe_domain.csv")

In [5]:
train = train.drop(['EXT_SOURCE_3_plus_EXT_SOURCE_2', 'EXT_SOURCE_1_plus_EXT_SOURCE_3', 'EXT_SOURCE_1_plus_EXT_SOURCE_2'], axis=1)
test = test.drop(['EXT_SOURCE_3_plus_EXT_SOURCE_2', 'EXT_SOURCE_1_plus_EXT_SOURCE_3', 'EXT_SOURCE_1_plus_EXT_SOURCE_2'], axis=1)

In [6]:
def xgboost(train, test):
    
    X = train.iloc[:,1:-1]
    y = train.iloc[:,-1]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

    model = xgb.XGBClassifier(tree_method="gpu_hist")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print("accuracy score", accuracy_score(y_test, y_pred))
    print("recall score", recall_score(y_test, y_pred))
    print("precision score", precision_score(y_test, y_pred))
    print("F1 score", f1_score(y_test, y_pred))

    X_train = train.iloc[:,1:-1]
    X_test = test.iloc[:,1:]
    y_train = train.iloc[:,-1]

    model = xgb.XGBClassifier(tree_method="gpu_hist")
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_test)

    # Because it's 2 dimentsional...
    y_pred = y_pred[:, 1]

    test_ids = test['SK_ID_CURR']
    submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': y_pred})

    return submission

In [7]:
xgboost(train, test).to_csv("Submission/xgboost_agg_fe_domain.csv", index=False)

accuracy score 0.9166646341959139
recall score 0.05604556221964619
precision score 0.3855755894590846
F1 score 0.09786578657865787
